1. How to represent single trace operators?
2. How to implement commutators of these

In [1]:
import numpy as np
import sympy as sp
from typing import Union, Self
from numbers import Number
from itertools import chain
from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem
from itertools import product

## One Matrix Model

In [2]:
class BootstrapSystem():
    '''
    Docstring
    '''
    def __init__(
            self,
            matrix_system: MatrixSystem,
            hamiltonian: SingleTraceOperator,
            gauge: MatrixOperator,
            half_max_degree: int
            ):
        self.matrix_system = matrix_system
        self.hamiltonian = hamiltonian
        self.gauge = gauge
        self.half_max_degree = half_max_degree
        self.operator_list = self.generate_operators(2*half_max_degree)
        if 2*self.half_max_degree < self.hamiltonian.max_degree:
            raise ValueError("2 * half_max_degree must be >= max degree of Hamiltonian.")

    def generate_operators(self, max_degree):
        operators = {deg: [x for x in product(self.matrix_system.operator_basis, repeat=deg)] for deg in range(0, max_degree+1)}
        self.psd_matrix_dim = sum(len(value) for degree, value in operators.items() if degree <= self.half_max_degree)
        return [x for xs in operators.values() for x in xs] # flatten

    def generate_hamiltonian_constraints(self):
        constaints = []
        for op_str in self.operator_list[0:self.psd_matrix_dim]:
             constaints.append(self.matrix_system.single_trace_commutator(
                 op1=self.hamiltonian,
                 op2=SingleTraceOperator(data={op_str: 1})
                 ))
        return constaints

    def generate_gauge_constraints(self):
        constaints = []
        for op_str in self.operator_list[0:self.psd_matrix_dim]:
             constaints.append(
                 (self.gauge * MatrixOperator(data={op_str: 1})).trace()
                 )
        return constaints


In [7]:
matrix_system = MatrixSystem(
    operator_basis=['X', 'P'],
    commutation_rules_concise = {
        ('P', 'X'): -1j,
    }
)

# scale variables as P = sqrt(N) P', X = sqrt(N) X'
hamiltonian = SingleTraceOperator(
        data={("P", "P"): 1, ("X", "X"): 1, ("X", "X", "X", "X"): 7}
    )

# <tr G O > = 0 might need to be applied only for O with deg <= L-2
gauge = MatrixOperator(data={('X', 'P'): 1j, ('P', 'X'): -1j, ():1})

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge=gauge,
    half_max_degree=3
)
bootstrap.operator_list[:bootstrap.psd_matrix_dim]

[(),
 ('X',),
 ('P',),
 ('X', 'X'),
 ('X', 'P'),
 ('P', 'X'),
 ('P', 'P'),
 ('X', 'X', 'X'),
 ('X', 'X', 'P'),
 ('X', 'P', 'X'),
 ('X', 'P', 'P'),
 ('P', 'X', 'X'),
 ('P', 'X', 'P'),
 ('P', 'P', 'X'),
 ('P', 'P', 'P')]

In [8]:
bootstrap.generate_hamiltonian_constraints()

[SingleTraceOperator(operators=[], coeff=[]),
 SingleTraceOperator(operators=[('P',)], coeff=[-2j]),
 SingleTraceOperator(operators=[('X',), ('X', 'X', 'X')], coeff=[2j, 28j]),
 SingleTraceOperator(operators=[('P', 'X'), ('X', 'P')], coeff=[-2j, -2j]),
 SingleTraceOperator(operators=[('P', 'P'), ('X', 'X'), ('X', 'X', 'X', 'X')], coeff=[-2j, 2j, 28j]),
 SingleTraceOperator(operators=[('P', 'P'), ('X', 'X'), ('X', 'X', 'X', 'X')], coeff=[-2j, 2j, 28j]),
 SingleTraceOperator(operators=[('X', 'P'), ('P', 'X'), ('X', 'X', 'X', 'P'), ('P', 'X', 'X', 'X')], coeff=[2j, 2j, 28j, 28j]),
 SingleTraceOperator(operators=[('P', 'X', 'X'), ('X', 'P', 'X'), ('X', 'X', 'P')], coeff=[-2j, -2j, -2j]),
 SingleTraceOperator(operators=[('P', 'X', 'P'), ('X', 'P', 'P'), ('X', 'X', 'X'), ('X', 'X', 'X', 'X', 'X')], coeff=[-2j, -2j, 2j, 28j]),
 SingleTraceOperator(operators=[('P', 'P', 'X'), ('X', 'P', 'P'), ('X', 'X', 'X'), ('X', 'X', 'X', 'X', 'X')], coeff=[-2j, -2j, 2j, 28j]),
 SingleTraceOperator(operator

In [9]:
bootstrap.generate_gauge_constraints()

[SingleTraceOperator(operators=[('X', 'P'), ('P', 'X'), ()], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X', 'P', 'X'), ('P', 'X', 'X'), ('X',)], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X', 'P', 'P'), ('P', 'X', 'P'), ('P',)], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X', 'P', 'X', 'X'), ('P', 'X', 'X', 'X'), ('X', 'X')], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X', 'P', 'X', 'P'), ('P', 'X', 'X', 'P'), ('X', 'P')], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X', 'P', 'P', 'X'), ('P', 'X', 'P', 'X'), ('P', 'X')], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X', 'P', 'P', 'P'), ('P', 'X', 'P', 'P'), ('P', 'P')], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X', 'P', 'X', 'X', 'X'), ('P', 'X', 'X', 'X', 'X'), ('X', 'X', 'X')], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X', 'P', 'X', 'X', 'P'), ('P', 'X', 'X', 'X', 'P'), ('X', 'X', 'P')], coeff=[1j, -1j, 1]),
 SingleTraceOperator(operators=[('X'

## OLD

In [ ]:
matrix_system = MatrixSystem(
    operator_basis=['X0', 'P1'],
    commutation_rules_concise = {
        ('P0', 'X0'): -1j,
        ('P1', 'X1'): -1j,
        ('P2', 'X2'): -1j,
    }
)
len(matrix_system.commutation_rules)